# Testing User cases in TLS
Test- Version 8.0.20-1

[Original HPCC TLS configuration tutorial](https://github.com/hpcc-systems/HPCC-Platform/blob/master/helm/examples/certmanager/README.md)
to:
* install cert manager
* Create a root certificate for HPCC local cluster certificate authority
* create kubernetes tls secret
## Prerequisites:
1. Install Helm 
2. Install Ingress Nginx controller
3. Deploy HPCC helm chart:

1. Install Helm:

Visit this [link](https://helm.sh/docs/intro/install/) to install the latest version of Helm.

2. Install Nginx controller with Helm

In [ ]:
helm repo add ingress-nginx https://kubernetes.github.io/ingress-nginx
helm repo update

helm install ingress-nginx ingress-nginx/ingress-nginx

3. Deploy the HPCC helm chart

set certificates to be enabled, and the version to be 8.0.20-1

In [ ]:
$ helm install hpcc hpcc/ --set global.image.version=8.0.20-1 --set certificates.enabled=true -f examples/azure/values-auto-azurefile.yaml

NAME: hpcc
LAST DEPLOYED: Mon Jul 12 14:29:33 2021
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Thank you for installing the HPCC chart.

This chart has defined the following HPCC components:
  dali.mydali
  dfuserver.dfuserver
  eclagent.hthor
  eclagent.roxie-workunit
  eclccserver.myeclccserver
  esp.eclwatch
  esp.eclservices
  esp.eclqueries
  esp.esdl-sandbox
  esp.sql2ecl
  roxie.roxie
  thor.thor
  sasha.dfurecovery-archiver
  sasha.dfuwu-archiver
  sasha.file-expiry
  sasha.wu-archiver


## Testing TLS without Ingress 
Before applying Ingress file, the eclwatch IP is not accessible from the webpage, run the command to test the TLS from the command line.

In [ ]:
$ ecl run -s 20.72.177.107 --ssl hthor examples/certmanager/localhttpcall.ecl

<Result>
<Dataset name='localHttpEchoResult'>
 <Row><Method>GET</Method><UrlPath>/WsSmc/HttpEcho</UrlPath><UrlParameters>name=doe,joe&amp;number=1</UrlParameters><Headers><Header>Accept-Encoding: gzip, deflate</Header><Header>Accept: text/xml</Header></Headers><Content></Content></Row>
</Dataset>
</Result>

## User case 1.  Adding Ingress in front of TLS secret
After using the following command, apply the ingress file to route to backend eclwatch.

In [ ]:
kubectl create secret tls hpcc-local-issuer-key-pair --cert=ca.crt --key=ca.key

In [ ]:
nano eclwatch.yaml

In [ ]:
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: eclwatch-ingress
  annotations:
    kubernetes.io/ingress.class: nginx
spec:
  rules:
  -  http:
      paths:
      - path: /
        pathType: Prefix
        backend:
         service:
           name: eclwatch
           port:
             number: 8010

Create the service with the following command:

In [ ]:
$ kubectl apply -f eclwatch.yaml
ingress.networking.k8s.io/eclwatch-ingress configured

### Test the Ingress route
To test the Ingress route, visit the External IP address for ECLwatch, followed by port number 8010.

Use the following command to get the External IP address:

In [1]:
kubectl get svc

NAME                                 TYPE           CLUSTER-IP     EXTERNAL-IP     PORT(S)                      AGE
cert-manager                         ClusterIP      10.0.43.14     <none>          9402/TCP                     9h
cert-manager-webhook                 ClusterIP      10.0.134.59    <none>          443/TCP                      9h
eclqueries                           LoadBalancer   10.0.44.101    20.72.178.0     8002:31283/TCP               9h
eclservices                          ClusterIP      10.0.223.39    <none>          8010/TCP                     9h
eclwatch                             LoadBalancer   10.0.190.82    20.72.177.107   8010:30484/TCP               9h
esdl-sandbox                         LoadBalancer   10.0.25.173    20.72.177.151   8899:30547/TCP               9h
ingress-nginx-controller             LoadBalancer   10.0.21.199    52.188.94.0     80:30397/TCP,443:31991/TCP   15h
ingress-nginx-controller-admission   ClusterIP      10.0.19.198    <none>     

The external IP is **20.72.177.107**.  Visit this webpage, enter the IP into a web browser followed by port number 8010.

https://20.72.177.107:8010

The result is the ECL watch webpage.

## Case 2. Using ssl redirect
After installing creating TLS secret and installing SSL, redirect visitors who are trying to access the HTTP site to the HTTPS site. Use the annotaitons below to redirect visitors to the HTTPS domain automatically, when they use HTTP domain.

Use backend protocol to indicate how Nginx should communicate with the backend service. (Replaces secure-backends in older versions) Valid Values: HTTP, HTTPS, GRPC, GRPCS, AJP and FCGI

Open the previously created file named **eclwatch.yaml**, and add the following annotations:

In [ ]:
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: eclwatch-ingress
  annotations:
    nginx.ingress.kubernetes.io/backend-protocol: "HTTPS"
    kubernetes.io/ingress.class: nginx
    nginx.ingress.kubernetes.io/ssl-redirect: "true"
spec:
  rules:
  -  http:
      paths:
      - path: /
        pathType: Prefix
        backend:
         service:
           name: eclwatch
           port:
             number: 8010

Delete the previously created file, then apply the new one:

In [ ]:
$ kubectl delete -f eclwatch.yaml
ingress.networking.k8s.io "eclwatch-ingress" deleted

$ kubectl apply -f eclwatch.yaml
ingress.networking.k8s.io/eclwatch-ingress created

## Test the annotations:
To test the annotations, use the eclwatch External IP, gotten previously from using the command **kubectl get svc**, and visit the IP for eclwatch, followed by :8010

20.72.177.107:8010


The annotations should automatically redirect domain to HTTPS, eclwatch webpage.